In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
path = r'/Users/kibbles/Documents/Metis/metisproject1/'
turnstiles_file = r'turnstiles_june2019.txt'
path+turnstiles_file

'/Users/kibbles/Documents/Metis/metisproject1/turnstiles_june2019.txt'

In [3]:
def processTurnstiles(path):
    '''
    Reads in MTA turnstile data from 'path,'
    Converts 'DATE' and 'TIME' cols to Datetime objects,
    if diff = False, adds 'ENTRIES_DIFF' col of count of entries (calculated 
    from 'ENTRIES' running total)
    
    Returns: pandas DataFrame
    '''
    weekdays = ['MON','TUE','WED','THU','FRI','SAT','SUN']
    
    df = pd.read_csv(path)
    df.reset_index(inplace=True)
    df.drop(columns='index', inplace=True)
    
    # Convert Date and Time to Datetime objects
    df['DATETIME']= pd.to_datetime(  df['DATE'] +' '+ df['TIME'] )
    df['DATE'] = pd.to_datetime(df['DATE'])
    # Add weekday column. DOF = "day of week"
    df['DOF'] = [weekdays[df['DATETIME'][1].weekday()] for dstring in df.DATE.tolist()]
    
    # Drop dupes
    df = df.drop_duplicates()
    
    # Create Station ID using Station Name and Line Name
    # sort each linename, since subway lines aren't listed in a consistent order
    lines = [''.join(sorted(line)) for line in df['LINENAME']] 
    df['STATION_ID']=df['STATION'] + "_" + pd.Series(lines)
    
    # Calculate Entry count from cumulative 'ENTRIES' column
    df['ENTRIES_DIFF']=( df.groupby(['STATION_ID','UNIT','SCP'],as_index=False)['ENTRIES']
                           .transform(pd.Series.diff)['ENTRIES']
                       )
    # Replace NaN/empties in 'ENTRIES_DIFF' with 0
    df.fillna(value={'ENTRIES_DIFF': 0}, inplace=True)
    
    # Convert 'ENTRIES_DIFF' to int()
    df = df.astype({'ENTRIES_DIFF': 'int32'}, copy=False)
    
    # Convert negative entries to positive values (from turnstiles counting down instead of up)
    df['ENTRIES_DIFF']=df['ENTRIES_DIFF'].abs()
    
    return df

Read in the turnstiles DataFrame using the above method

In [4]:
df = processTurnstiles(path+turnstiles_file)

In [5]:
# Line to save df as csv file, just in case
# df.to_csv(r'/Users/kibbles/Documents/Metis/metisproject1/turnstiles_june2019_procd.txt', index=False, header=True)

# Data info and description

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 823385 entries, 0 to 823384
Data columns (total 16 columns):
 #   Column                                                                Non-Null Count   Dtype         
---  ------                                                                --------------   -----         
 0   level_0                                                               823385 non-null  int64         
 1   C/A                                                                   823385 non-null  object        
 2   UNIT                                                                  823385 non-null  object        
 3   SCP                                                                   823385 non-null  object        
 4   STATION                                                               823385 non-null  object        
 5   LINENAME                                                              823385 non-null  object        
 6   DIVISION                    

In [7]:
df.describe()

,level_0,ENTRIES,EXITS,ENTRIES_DIFF
count,823385.000000,8.233850e+05,8.233850e+05,8.233850e+05
mean,411692.000000,4.202873e+07,3.436267e+07,9.764691e+03
std,237690.920036,2.135794e+08,1.964719e+08,3.166091e+06
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,205846.000000,2.923970e+05,1.147550e+05,1.000000e+01
50%,411692.000000,2.134706e+06,1.221939e+06,7.400000e+01
75%,617538.000000,6.746517e+06,4.594824e+06,2.400000e+02
max,823384.000000,2.129277e+09,2.124038e+09,1.819016e+09


In [8]:
df.head()

,level_0,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DOF,STATION_ID,ENTRIES_DIFF
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-06-01,00:00:00,REGULAR,7080105,2398899,2019-06-01 00:00:00,SAT,59 ST_456NQRW,0
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-06-01,04:00:00,REGULAR,7080137,2398904,2019-06-01 04:00:00,SAT,59 ST_456NQRW,32
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-06-01,08:00:00,REGULAR,7080161,2398937,2019-06-01 08:00:00,SAT,59 ST_456NQRW,24
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-06-01,12:00:00,REGULAR,7080276,2399025,2019-06-01 12:00:00,SAT,59 ST_456NQRW,115
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-06-01,16:00:00,REGULAR,7080520,2399084,2019-06-01 16:00:00,SAT,59 ST_456NQRW,244


# Check the data for outliers

In [9]:
max_table = df.sort_values(['ENTRIES_DIFF']).groupby(['STATION_ID','SCP'])['STATION_ID','ENTRIES_DIFF'].max()

/Users/kibbles/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


Let's look at the top ten most trafficked stations. What's going on here?

In [10]:
# max_table[max_table['ENTRIES_DIFF'] > 100000 ].sort_values(['ENTRIES_DIFF'], ascending=False).head(10)
max_table.sort_values(['ENTRIES_DIFF'], ascending=False).head(10)

STATION_ID  \
STATION_ID                   SCP                                      
167 ST_BD                    00-00-00                     167 ST_BD   
3 AV-149 ST_25               00-00-00                3 AV-149 ST_25   
                             01-00-02                3 AV-149 ST_25   
52 ST_7                      00-06-00                       52 ST_7   
3 AV-149 ST_25               00-00-01                3 AV-149 ST_25   
14 ST_123FLM                 02-00-01                  14 ST_123FLM   
MOSHOLU PKWY_4               00-06-01                MOSHOLU PKWY_4   
SPRING ST_CE                 01-06-01                  SPRING ST_CE   
42 ST-PORT AUTH_1237ACENQRSW 00-06-01  42 ST-PORT AUTH_1237ACENQRSW   
8 AV_ACEL                    00-06-00                     8 AV_ACEL   

                                       ENTRIES_DIFF  
STATION_ID                   SCP                     
167 ST_BD                    00-00-00    1819016355  
3 AV-149 ST_25               00-00-00    1169826812  
                             01-00-02    1061119023  
52 ST_7                      00-06-00     867827527  
3 AV-149 ST_25               00-00-01     701922569  
14 ST_123FLM                 02-00-01     621484858  
MOSHOLU PKWY_4               00-06-01     451162879  
SPRING ST_CE                 01-06-01     117390566  
42 ST-PORT AUTH_1237ACENQRSW 00-06-01      99037371  
8 AV_ACEL                    00-06-00      56432969

What! 1.8 BILLION entries for '167_BD'?! Let's take a look at the individual SCP's for this station

In [11]:
df[df['STATION_ID'] == '167 ST_BD'].groupby(['SCP']).sum()

,level_0,ENTRIES,EXITS,ENTRIES_DIFF
SCP,,,,
00-00-00,64530097,159890454407,159691218272,1819051350
00-00-01,64610193,1503951251,777876284,48140
00-00-02,64617419,302592231997,145585849801,37889
00-00-03,64624645,1704836619,686574479,46358
01-00-00,64097794,11270330,8773351,13739
01-00-01,64104935,10974817,2989421,13329
01-00-02,64112076,24275599,4342679,26349


Whoa! It seems like the culprit at '167 ST_BD' is one particular SCP. Let's drill down farther - maybe if we sort by ENTRIES_DIFF we can find the exact date when it happened.

In [12]:
( df[ (df['STATION_ID'] == '167 ST_BD') & (df['SCP'] == '00-00-00') ]
 [['STATION_ID','SCP','ENTRIES','DATETIME','ENTRIES_DIFF']]
 .sort_values(['ENTRIES_DIFF'], ascending=False)
 .head(3)
)

,STATION_ID,SCP,ENTRIES,DATETIME,ENTRIES_DIFF
277972,167 ST_BD,00-00-00,1828704206,2019-06-14 16:00:00,1819016355
483779,167 ST_BD,00-00-00,1828696773,2019-06-19 12:00:00,578
690291,167 ST_BD,00-00-00,1828688443,2019-06-24 12:00:00,575


Looking at the first row of the table above, the problem seems to be one particular datetime entry:

'2019-06-14 16:00:00' at location 277972.

Let's see if we can take a look at that row and it's surroundings.

In [13]:
df[['STATION_ID','SCP','ENTRIES','DATETIME','ENTRIES_DIFF']].loc[277968:277975]

,STATION_ID,SCP,ENTRIES,DATETIME,ENTRIES_DIFF
277968,167 ST_BD,00-00-00,9687851,2019-06-14 00:00:00,0
277969,167 ST_BD,00-00-00,9687851,2019-06-14 04:00:00,0
277970,167 ST_BD,00-00-00,9687851,2019-06-14 08:00:00,0
277971,167 ST_BD,00-00-00,9687851,2019-06-14 12:00:00,0
277972,167 ST_BD,00-00-00,1828704206,2019-06-14 16:00:00,1819016355
277973,167 ST_BD,00-00-00,1828703861,2019-06-14 20:00:00,345
277974,167 ST_BD,00-00-01,8832140,2019-06-08 00:00:00,229
277975,167 ST_BD,00-00-01,8832203,2019-06-08 04:00:00,63


Ah. Seems like the turnstile wasn't recording any entries, then the counter went wild and eight hours later the counter changed yet again.

So IMHO it might be best to these outlier values to zero. We'll use the turnstile's standard deviation to measure if a particular ENTRIES_DIFF count is faaar out, man.

In [14]:
# Remove Outliers
df = df[df.groupby(['STATION_ID','SCP'])['ENTRIES_DIFF'].apply(lambda x: np.abs(x - x.mean()) / x.std() < 3)]

Let's look at the maxtable again

In [17]:
max_table = df.sort_values(['ENTRIES_DIFF']).groupby(['STATION_ID','SCP'])['STATION_ID','ENTRIES_DIFF'].max()
max_table.sort_values(['ENTRIES_DIFF'], ascending=False).head(10)

/Users/kibbles/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,,STATION_ID,ENTRIES_DIFF
STATION_ID,SCP,,
NEWARK BM BW_1,01-01-03,NEWARK BM BW_1,22392
PATH NEW WTC_1,00-01-05,PATH NEW WTC_1,20747
THIRTY THIRD ST_1,01-00-07,THIRTY THIRD ST_1,8705
PATH NEW WTC_1,00-01-00,PATH NEW WTC_1,6808
14TH STREET_1,00-00-02,14TH STREET_1,6727
NEWARK BM BW_1,01-00-08,NEWARK BM BW_1,6116
PATH NEW WTC_1,00-02-07,PATH NEW WTC_1,5472
THIRTY THIRD ST_1,01-00-05,THIRTY THIRD ST_1,4951
14TH STREET_1,00-00-00,14TH STREET_1,4797


Much better! These Entry counts look more realistic.